In [31]:
import pandas as pd


In [32]:
import numpy as np

In [33]:
!pip install pymongo


In [34]:
from pymongo import MongoClient

In [35]:
client = MongoClient("localhost",27017)

In [36]:
ratings = pd.read_csv("Data/ratings.dat", sep="::", header=None, names=["user_id", "movie_id", "rating", "timestamp"])

C:\Users\chittithalli\AppData\Local\Temp\ipykernel_14176\776172798.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("Data/ratings.dat", sep="::", header=None, names=["user_id", "movie_id", "rating", "timestamp"])


In [37]:
ratings.drop(axis=1, labels=["timestamp"], inplace=True)

In [38]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [39]:
ratings.dtypes

user_id     int64
movie_id    int64
rating      int64
dtype: object

In [40]:
ratings = ratings.applymap(str)

C:\Users\chittithalli\AppData\Local\Temp\ipykernel_14176\20016063.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ratings = ratings.applymap(str)


In [41]:
db = client.movrec

In [42]:
ratings[ratings["movie_id"]=="858"].head()

,user_id,movie_id,rating
998,10,858,3
1347,12,858,5
2376,19,858,4
2684,22,858,4
2984,23,858,5


In [43]:
movie_ratings = dict()

In [44]:
for i in ratings.index:
    try:
        movie_ratings[ratings.loc[i, "movie_id"]].append({ratings.loc[i, "user_id"]: ratings.loc[i, "rating"]})
    except KeyError:  # Catch KeyError explicitly
        movie_ratings[ratings.loc[i, "movie_id"]] = [{ratings.loc[i, "user_id"]: ratings.loc[i, "rating"]}]


In [45]:
movie_ratings["858"]

[{'10': '3'},
 {'12': '5'},
 {'19': '4'},
 {'22': '4'},
 {'23': '5'},
 {'24': '5'},
 {'27': '5'},
 {'29': '5'},
 {'33': '5'},
 {'36': '5'},
 {'42': '3'},
 {'44': '3'},
 {'47': '4'},
 {'48': '4'},
 {'49': '5'},
 {'53': '5'},
 {'58': '5'},
 {'62': '5'},
 {'74': '4'},
 {'88': '5'},
 {'90': '5'},
 {'97': '5'},
 {'100': '4'},
 {'104': '5'},
 {'105': '5'},
 {'107': '4'},
 {'116': '4'},
 {'120': '5'},
 {'122': '3'},
 {'123': '5'},
 {'126': '4'},
 {'127': '4'},
 {'129': '5'},
 {'130': '4'},
 {'131': '4'},
 {'136': '4'},
 {'138': '5'},
 {'140': '5'},
 {'146': '5'},
 {'148': '4'},
 {'149': '5'},
 {'151': '4'},
 {'154': '3'},
 {'155': '4'},
 {'157': '5'},
 {'158': '5'},
 {'163': '4'},
 {'164': '4'},
 {'169': '4'},
 {'170': '5'},
 {'171': '4'},
 {'173': '5'},
 {'174': '5'},
 {'187': '4'},
 {'188': '1'},
 {'190': '5'},
 {'192': '5'},
 {'195': '5'},
 {'198': '4'},
 {'204': '5'},
 {'205': '5'},
 {'216': '3'},
 {'225': '4'},
 {'231': '5'},
 {'232': '5'},
 {'234': '4'},
 {'235': '5'},
 {'238': '4'},
 {

In [46]:
movieratings = []
for i,j in movie_ratings.items():
    movieratings.append({"movie_id":i, "ratings":j})

In [47]:
type(movieratings)

list

In [48]:
movieratings[:10]

[{'movie_id': '1193',
  'ratings': [{'1': '5'},
   {'2': '5'},
   {'12': '4'},
   {'15': '4'},
   {'17': '5'},
   {'18': '4'},
   {'19': '5'},
   {'24': '5'},
   {'28': '3'},
   {'33': '5'},
   {'39': '5'},
   {'42': '3'},
   {'44': '4'},
   {'47': '4'},
   {'48': '4'},
   {'49': '4'},
   {'53': '5'},
   {'54': '5'},
   {'58': '5'},
   {'59': '4'},
   {'62': '4'},
   {'80': '4'},
   {'81': '5'},
   {'88': '5'},
   {'89': '5'},
   {'95': '5'},
   {'96': '3'},
   {'99': '2'},
   {'102': '5'},
   {'104': '2'},
   {'107': '5'},
   {'109': '2'},
   {'120': '5'},
   {'123': '5'},
   {'131': '5'},
   {'136': '4'},
   {'139': '4'},
   {'146': '4'},
   {'150': '5'},
   {'151': '4'},
   {'155': '5'},
   {'166': '5'},
   {'169': '5'},
   {'175': '5'},
   {'178': '4'},
   {'180': '5'},
   {'190': '5'},
   {'191': '3'},
   {'192': '4'},
   {'195': '5'},
   {'196': '4'},
   {'202': '4'},
   {'206': '1'},
   {'208': '5'},
   {'216': '4'},
   {'224': '4'},
   {'225': '4'},
   {'227': '4'},
   {'230': 

In [49]:
result = db.MovieRatings.insert_many(movieratings)

In [50]:
len(movieratings[1]["ratings"])

525

In [51]:
records = ratings.to_dict(orient="records")

In [52]:
records[0:5]

[{'user_id': '1', 'movie_id': '1193', 'rating': '5'},
 {'user_id': '1', 'movie_id': '661', 'rating': '3'},
 {'user_id': '1', 'movie_id': '914', 'rating': '3'},
 {'user_id': '1', 'movie_id': '3408', 'rating': '4'},
 {'user_id': '1', 'movie_id': '2355', 'rating': '5'}]

In [53]:
result = db.Ratings.insert_many(records)

In [54]:
movies = pd.read_csv("Data/movies.dat", sep=";", index_col=0, names=["title", "genres", "movie_id", "imdb_id", "year"], skiprows=1, dtype={"imdb_id":str})

In [55]:
movies.head()

,title,genres,movie_id,imdb_id,year
1,Toy Story,Animation|Children's|Comedy,1,114709,1995
2,Jumanji,Adventure|Children's|Fantasy,2,113497,1995
3,Grumpier Old Men,Comedy|Romance,3,113228,1995
4,Waiting to Exhale,Comedy|Drama,4,114885,1995
5,Father of the Bride Part II,Comedy,5,113041,1995


In [56]:
movies = movies.applymap(str)

C:\Users\chittithalli\AppData\Local\Temp\ipykernel_14176\1156796315.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  movies = movies.applymap(str)


In [57]:
movies = movies.to_dict(orient="records")

In [58]:
movies[:5]

[{'title': 'Toy Story ',
  'genres': "Animation|Children's|Comedy",
  'movie_id': '1',
  'imdb_id': '114709',
  'year': '1995'},
 {'title': 'Jumanji ',
  'genres': "Adventure|Children's|Fantasy",
  'movie_id': '2',
  'imdb_id': '113497',
  'year': '1995'},
 {'title': 'Grumpier Old Men ',
  'genres': 'Comedy|Romance',
  'movie_id': '3',
  'imdb_id': '113228',
  'year': '1995'},
 {'title': 'Waiting to Exhale ',
  'genres': 'Comedy|Drama',
  'movie_id': '4',
  'imdb_id': '114885',
  'year': '1995'},
 {'title': 'Father of the Bride Part II ',
  'genres': 'Comedy',
  'movie_id': '5',
  'imdb_id': '113041',
  'year': '1995'}]

In [59]:
for i in range(len(movies)):
    movies[i]["genres"] = movies[i]["genres"].split("|")

In [60]:
movies[:5]

[{'title': 'Toy Story ',
  'genres': ['Animation', "Children's", 'Comedy'],
  'movie_id': '1',
  'imdb_id': '114709',
  'year': '1995'},
 {'title': 'Jumanji ',
  'genres': ['Adventure', "Children's", 'Fantasy'],
  'movie_id': '2',
  'imdb_id': '113497',
  'year': '1995'},
 {'title': 'Grumpier Old Men ',
  'genres': ['Comedy', 'Romance'],
  'movie_id': '3',
  'imdb_id': '113228',
  'year': '1995'},
 {'title': 'Waiting to Exhale ',
  'genres': ['Comedy', 'Drama'],
  'movie_id': '4',
  'imdb_id': '114885',
  'year': '1995'},
 {'title': 'Father of the Bride Part II ',
  'genres': ['Comedy'],
  'movie_id': '5',
  'imdb_id': '113041',
  'year': '1995'}]

In [61]:
result = db.Movies.insert_many(movies)

In [62]:
movie = list(db.Movies.find({"movie_id":"111"}))[0]

In [63]:
movie["title"]

'Taxi Driver '

In [64]:
for g in movie["genres"]:
    print(g)

Drama
Thriller


In [66]:
#ratings[ratings.movie_id == ]
filtered_ratings = ratings[ratings.movie_id == 1]


In [67]:
print(ratings["movie_id"].isnull().sum())


0


In [69]:
print(type(movies))


<class 'list'>


In [70]:
import pandas as pd

# Assuming movies is a list of dictionaries
movies_df = pd.DataFrame(movies)
print(movies_df.head())


                          title                            genres movie_id  \
0                    Toy Story    [Animation, Children's, Comedy]        1   
1                      Jumanji   [Adventure, Children's, Fantasy]        2   
2             Grumpier Old Men                  [Comedy, Romance]        3   
3            Waiting to Exhale                    [Comedy, Drama]        4   
4  Father of the Bride Part II                           [Comedy]        5   

  imdb_id  year                       _id  
0  114709  1995  6740405a78e5f939845d0672  
1  113497  1995  6740405a78e5f939845d0673  
2  113228  1995  6740405a78e5f939845d0674  
3  114885  1995  6740405a78e5f939845d0675  
4  113041  1995  6740405a78e5f939845d0676  


In [71]:
movies.head()

AttributeError: 'list' object has no attribute 'head'

In [73]:
print(movies[:5])  # Displays the first 5 elements of the list



[{'title': 'Toy Story ', 'genres': ['Animation', "Children's", 'Comedy'], 'movie_id': '1', 'imdb_id': '114709', 'year': '1995', '_id': ObjectId('6740405a78e5f939845d0672')}, {'title': 'Jumanji ', 'genres': ['Adventure', "Children's", 'Fantasy'], 'movie_id': '2', 'imdb_id': '113497', 'year': '1995', '_id': ObjectId('6740405a78e5f939845d0673')}, {'title': 'Grumpier Old Men ', 'genres': ['Comedy', 'Romance'], 'movie_id': '3', 'imdb_id': '113228', 'year': '1995', '_id': ObjectId('6740405a78e5f939845d0674')}, {'title': 'Waiting to Exhale ', 'genres': ['Comedy', 'Drama'], 'movie_id': '4', 'imdb_id': '114885', 'year': '1995', '_id': ObjectId('6740405a78e5f939845d0675')}, {'title': 'Father of the Bride Part II ', 'genres': ['Comedy'], 'movie_id': '5', 'imdb_id': '113041', 'year': '1995', '_id': ObjectId('6740405a78e5f939845d0676')}]


In [74]:
ratings.dtypes

user_id     object
movie_id    object
rating      object
dtype: object

In [76]:
#movies.dtypes

In [77]:
#for i in 

In [78]:
import pandas as pd

# Example: Converting a list of dictionaries to a DataFrame
movies_df = pd.DataFrame(movies)

# Check the data types of the columns
print(movies_df.dtypes)


title       object
genres      object
movie_id    object
imdb_id     object
year        object
_id         object
dtype: object
